In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
from hanspell import spell_checker
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgbm
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import confusion_matrix, accuracy_score
import re
from gensim import corpora
from gensim.models.fasttext import FastText

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [26]:
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences

In [116]:
#import tensorflow as tf

In [4]:
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic/')

In [5]:
df = pd.read_csv('../Modeling/train/youtube_topic_category_train_211216_2.csv',encoding='utf-8-sig')
df

,video_id,title,topic_label,topic_category
0,yeLDYfGHoD4,[아이맥/아이폰13/갤럭시Z폴드2 등등?!] #Shorts,0,가전제품
1,PYt36zFuPNs,"""10만원 이하로 살 수 있는 최강 가성비"" 모든 아이폰 중고 시세 1부 | 3GS...",0,가전제품
2,WESnBkmPolM,"""BTS 내 Flip 안에 저-장????"" 갤럭시Z 플립3, 외부화면 설정해봤다",0,가전제품
3,gijBNRvdG4A,"""가격 올리려고 새 모델 출시한 건 아니겠지?"" 소니 무선마이크 SONY UWP-D21",0,가전제품
4,p1mwr7g7oWM,"""갤럭시 왜 써요..?"" 사람들에게 갤럭시 스마트폰 쓰는 이유 물어봤습니다.",0,가전제품
...,...,...,...,...
203767,8gt15xGPw8I,부정경선 전면무효 여의도 국힘당사 개새끼들 10월 15일,17,F등급
203768,q1s8izMablI,절대19금 분명말씀드림. 성인용품임.,17,F등급
203769,1Zc3R0Wvu2Y,하지말라! 하지마! 이 개새X야!,17,F등급
203770,9b5kciewBgs,졸라 맛있어보여서 입에 침이 고일 때 원어민들은 뭐라고 할까,17,F등급


In [6]:
df['topic_label'].value_counts()

5     21078
7     20662
14    19435
15    15553
16    14926
10    14539
2     13634
4     12740
1     11700
11    11677
0     10054
8      8705
3      8332
13     6660
6      6422
9      5049
12     2586
17       20
Name: topic_label, dtype: int64

In [7]:
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
 
    return text

In [8]:
SELECT_POS = ['NNG','NNP','XR','SL','SN','VV','VA','ETN','NA','NR','XSN','UNKNOWN']

def pos_tagging(text):
    word_list = []
    for word in mecab.pos(text):
        if word[1] in SELECT_POS:
            word_list.append(word[0])
    return word_list

In [9]:
df = df[df['title'].str.len()>11]
df.reset_index(drop=True,inplace=True)
df['text'] = df['title'].apply(lambda x : mecab.morphs(x))

<ipython-input-9-52d808de1435>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['title'].apply(lambda x : mecab.morphs(x))


In [10]:
df = df.sample(frac=1,random_state=0).reset_index(drop=True)
df['text2'] = df['text'].apply(lambda x : " ".join(x))
df

,video_id,title,topic_label,topic_category,text,text2
0,ogXonrXN694,자막) 우연히 봤다가 결말이 진짜 궁금해서 끝까지 본 그 영화 (영화리뷰 결말포함),10,미디어 콘텐츠,"[자막, ), 우연히, 봤, 다가, 결말, 이, 진짜, 궁금, 해서, 끝, 까지, ...",자막 ) 우연히 봤 다가 결말 이 진짜 궁금 해서 끝 까지 본 그 영화 ( 영화 리...
1,5TM61G_faK0,[휴넷 HRD 리더스 포럼 2022] 세션2_마이크로러닝｜칼라토거슨 전략실장,0,가전제품,"[[, 휴, 넷, HRD, 리더스, 포럼, 2022, ], 세션, 2, _, 마이크...",[ 휴 넷 HRD 리더스 포럼 2022 ] 세션 2 _ 마이크 로 러닝 ｜ 칼 라토...
2,IbtrjWO_7lY,파워뱅크 캠핑용품~! | 테슬라도 충전하는 끝판왕! | 지금까지 이런 제품은 없었다...,1,자동차와 오토바이,"[파워, 뱅크, 캠핑, 용품, ~!, |, 테슬라, 도, 충전, 하, 는, 끝판, ...",파워 뱅크 캠핑 용품 ~! | 테슬라 도 충전 하 는 끝판 왕 ! | 지금 까지 이...
3,t2rxpfUosXM,맛집탐방 기로의 인생 문어맛집 소개합니다 (+ 서울 문어맛집 2군데 더 소개),15,음식,"[맛집, 탐방, 기로, 의, 인생, 문어, 맛집, 소개, 합니다, (, +, 서울,...",맛집 탐방 기로 의 인생 문어 맛집 소개 합니다 ( + 서울 문어 맛집 2 군데 더...
4,bOxf1m6IoS4,HIM 온라인 영성학교 2단계 사????? ????? 1??? - 체안,16,종교,"[HIM, 온라인, 영성, 학교, 2, 단계, 사, ?, ????, ?, ????,...",HIM 온라인 영성 학교 2 단계 사 ? ???? ? ???? 1 ? ?? - 체 안
...,...,...,...,...,...,...
203580,vUxpReM9ycU,10분이면 반죽 끝~ 초간단 통밀스콘 만들기!! 에어프라이어 통밀스콘&바질스콘 레시피,15,음식,"[10, 분, 이, 면, 반죽, 끝, ~, 초, 간단, 통밀, 스콘, 만들, 기, ...",10 분 이 면 반죽 끝 ~ 초 간단 통밀 스콘 만들 기 ! ! 에어 프라이어 통밀...
203581,GoPgSe2z9S4,한국무용 중급자들을 위한 숙련된 무용강의 //남화연시나위춤-동살풀이1//Korea ...,9,예술과 인문학,"[한국, 무용, 중, 급자, 들, 을, 위한, 숙련, 된, 무용, 강의, /, /,...",한국 무용 중 급자 들 을 위한 숙련 된 무용 강의 / / 남화연 시나위 춤 - 동...
203582,VCj0qf5JulU,"(초간단 레시피) 따끈한 가을 간식 추천! 고구마와 크림의 환상적인 조합, 고구마 크림",15,음식,"[(, 초, 간단, 레시피, ), 따끈, 한, 가을, 간식, 추천, !, 고구마, ...",( 초 간단 레시피 ) 따끈 한 가을 간식 추천 ! 고구마 와 크림 의 환상 적 인...
203583,u-A26OTV5ug,"""합격할수이쒀"" 메이크업 국가 자격증 실기 제 3과제 레오파드 메이크업 | How ...",4,뷰티패션,"["", 합격, 할, 수, 이, 쒀, "", 메이크업, 국가, 자격증, 실기, 제, 3...",""" 합격 할 수 이 쒀 "" 메이크업 국가 자격증 실기 제 3 과제 레오 파드 메이크..."


In [16]:
ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    vector_size=100, # embedding dimension (default)
    window=5, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=5, # only consider tokens with at least n occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    min_n=2, # min character n-gram
    max_n=5 # max character n-gram
)

ft_model.build_vocab(df['text'])

ft_model.train(
    df['text'],
    epochs=5,
    total_examples=ft_model.corpus_count, 
    total_words=ft_model.corpus_total_words)

ft_model.save('YT_category_fasttext.model')

In [91]:
ft_model = FastText.load('YT_fasttext.model')

In [92]:
len(ft_model.wv)

118026

In [93]:
# create a tokenizer 
token = Tokenizer()
token.fit_on_texts(df['text'])
word_index = token.word_index


In [94]:
df_train = df[:180000]
df_test = df[180000:]
x_train = df_train['text']
x_valid = df_test['text']

In [95]:
y_train = df_train['topic_label']
y_valid = df_test['topic_label']

In [96]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
x_train_seq = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=50)
x_valid_seq = sequence.pad_sequences(token.texts_to_sequences(x_valid), maxlen=50)
# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 100))
words = []

In [97]:
for word, i in tqdm(word_index.items()):
    embedding_vector = ft_model.wv.get_vector(word) #embeddings_index.get(word)
    words.append(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|███████████████████████████████████████████████████████████████████████| 104135/104135 [00:01<00:00, 95711.83it/s]


In [98]:
embedded = keras.layers.Embedding(len(word_index)+1, 100, weights=[embedding_matrix], trainable=False)

In [99]:
model = Sequential()
# Keras Embedding layer with Word2Vec weights initialization
model.add(embedded)
model.add(Bidirectional(LSTM(200, return_sequences = True)))
model.add(GlobalMaxPool1D()) #풀링 : 과적합 방지
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.05)) 
model.add(Dense(18, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [100]:
history = model.fit(x_train_seq, y_train, epochs=5, batch_size=64,
                    validation_data=(x_valid_seq, y_valid), verbose=1)

Epoch 1/5
2813/2813 [==============================] - 34s 12ms/step - loss: 0.3052 - accuracy: 0.9180 - val_loss: 0.1253 - val_accuracy: 0.9663
Epoch 2/5
2813/2813 [==============================] - 32s 11ms/step - loss: 0.1065 - accuracy: 0.9716 - val_loss: 0.1050 - val_accuracy: 0.9715
Epoch 3/5
2813/2813 [==============================] - 32s 11ms/step - loss: 0.0751 - accuracy: 0.9798 - val_loss: 0.0811 - val_accuracy: 0.9788
Epoch 4/5
2813/2813 [==============================] - 32s 11ms/step - loss: 0.0561 - accuracy: 0.9850 - val_loss: 0.0703 - val_accuracy: 0.9819
Epoch 5/5
2813/2813 [==============================] - 32s 11ms/step - loss: 0.0428 - accuracy: 0.9881 - val_loss: 0.0724 - val_accuracy: 0.9818


## TEST DATA 불러오기

In [32]:
import pymysql
from pandas.io.json import json_normalize

In [34]:
conn  = pymysql.connect(host=host,port=port,user=user,passwd=passwd,db=db)  #데이터베이스 연결
cursor = conn.cursor()
conn.set_charset('utf8')

In [35]:
cursor.execute("Select * FROM youtube_video_lists") #From 367
columns = [item[0] for item in cursor.description]
data = pd.DataFrame(cursor.fetchall(),columns=columns)

In [36]:
data = data.dropna(subset=['title'])

### TEST DATA 전처리

In [38]:
data.reset_index(drop=True,inplace=True)
data['text'] = data['title'].apply(lambda x : mecab.morphs(x))

In [40]:
x_test = data['text']
x_test_seq = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=50)

In [101]:
y = model.predict(x_test_seq)

In [102]:
y

array([[3.2037216e-05, 2.3542553e-04, 6.3936459e-05, ..., 1.6878042e-04,
        3.7641529e-04, 9.9766430e-06],
       [1.7019197e-04, 5.7752355e-04, 2.4566753e-04, ..., 6.0667295e-04,
        2.6895122e-03, 6.3281252e-05],
       [3.2534369e-04, 3.3231687e-03, 2.1576521e-04, ..., 5.5762997e-04,
        7.4151084e-03, 1.3550635e-03],
       ...,
       [4.0944768e-05, 1.4264032e-04, 1.7837752e-05, ..., 2.1844509e-04,
        2.2149301e-05, 1.4622157e-05],
       [4.6719713e-04, 5.0420111e-01, 7.2946824e-02, ..., 1.4677242e-02,
        3.5600238e-03, 4.1787565e-02],
       [1.1395683e-02, 1.5093643e-03, 1.0751908e-03, ..., 1.9124338e-01,
        1.5744446e-03, 6.0524803e-04]], dtype=float32)

In [103]:
data['predict'] =  pd.DataFrame(y).idxmax(axis="columns").to_list() #LGBM

In [104]:
data

,video_id,title,channel_id,views,likes,dislikes,comments,tags,video_description,video_thumbnails_url,publishDate,duration,text,predict
0,___CMA1pEXA,[DOJIN] SUNMI선미 - LALALAY날라리 / Dance Cover / M...,UC12cVwwjEC1CG4Y8abhr9Xg,692267,29178,254,960,"도진 댄스커버, Cover Dance, LALALAY Cover Dance, 선미 ...",문의 메일✉ dojin@sandboxnetwork.net\n\nInstagram I...,,2019-09-02,None,"[[, DOJIN, ], SUNMI, 선미, -, LALALAY, 날라리, /, D...",7
1,___m5buSuTk,장윤정 족저근막염 노화라네 _ 해방타운,UCcRy0bDlIh0sKWiOv7XWQmA,11441,176,7,6,"장윤정, 하영이, 도하영, 슈퍼맨이 돌아왔다, 해방타운, jtbc, 노래가좋아, 도...",None,,2021-10-02,None,"[장윤정, 족저, 근막염, 노화, 라네, _, 해방, 타운]",13
2,___M9uxeTPo,[팩맨논평] 이제 몇 만원으로 안돼요. 더 주셔야 해요,UCksyDD1Gv6wmpyXJezZkVXw,39713,5073,63,367,"통신비지원, 선별지원",한국 국민들이 언제부터 재정적자.세금 걱정을 했을까요?\n국가부채 100조는 차세대...,,2020-09-12,None,"[[, 팩맨, 논평, ], 이제, 몇, 만, 원, 으로, 안, 돼요, ., 더, 주...",11
3,___pCwlibvw,이걸 본 사람들은 마치 동화 같은 이야기라며 감동했다 [작은영웅],UCSuuVcF9lix5dwFmAPkHE-g,6315,390,1,44,"작은영웅, 아직 살만한 세상, 감동실화, 감동적인 이야기, 감동적인 동영상, 감동영...",버스에 있던 사람들이 갑자기 너도나도 휴대폰을 꺼내더니 창밖을 보며 사진을 찍기 시...,,2021-07-28,None,"[이걸, 본, 사람, 들, 은, 마치, 동화, 같, 은, 이야기, 라며, 감동, 했...",9
4,__-5OLdwYaw,한국 레전드 락발라드 계보 4부 #JayTV,UCeXeCiZZ5OtiBTTn7UNN-cQ,14742,239,0,102,"jay tv, Jay TV, 추억 중독자, 한국 레전드, 토토가, 슈가맨, 90년대...",특집 기획 - 한국 락발라드 계보 4부 (락발라드 2차 부흥기 - 2대 황제 등장)...,,2021-08-12,None,"[한국, 레전드, 락, 발라드, 계보, 4, 부, #, JayTV]",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367229,ZzZyzYp8m9w,"[직캠] 191127 지창욱 베트남 인천공항 입국 (jichangwook,池昌旭) ...",UCAnRSFRuxYU087HNOGta3BA,82305,834,13,50,,"link O, re-upload X, edit X, crop logo X\n원주소 ...",,2019-11-28,None,"[[, 직, 캠, ], 191127, 지창욱, 베트남, 인천, 공항, 입국, (, ...",2
1367230,ZzZZEbhJOi8,Superheroes Dance 슈퍼히어로가 신나게 춤을 춘다고? 색칠공부 Hulk...,UCe8v7Loack1KSmvCpDhVv1w,19225,57,29,0,"pretend play, superhero, 슈퍼히어로, 헐크, 스파이더맨, 타노스...",#수아튜브\n무료 구독하기:https://bit.ly/2QwRt6b,,2020-09-03,None,"[Superheroes, Dance, 슈퍼, 히어로, 가, 신나, 게, 춤, 을, ...",8
1367231,zzzzjjJ69Ew,"프로 골퍼들과 9홀 대결, 과연 승자는? (feat. 시크릿 선물) ㅣ 9 hole...",UCRwrZJ9OCEIvdXFz0AZ5KNw,8396,574,11,97,"방성훈, 성훈, 골프, 남자골프, kpga, KPGA, 박정환, 정환프로, 고석완,...",02:47 미스터드라이버샷 어워드 Mr. Driver Shot Award\n07:...,,2021-09-07,None,"[프로, 골퍼, 들, 과, 9, 홀, 대결, ,, 과연, 승자, 는, ?, (, f...",8
1367232,zZzZoEbxZK0,파츠소분하고 비즈키트 제작하는 쇼핑몰 브이로그 | 슬라임 고수님들 도와주세요…,UCmxawXaLxT08isee1XTPXUg,1620,21,1,7,"비즈악세사리, 비즈팔찌, 비즈반지, 비즈목걸이, 초보사장브이로그, 동대문부자재상가,...",★ Instagram : @beauty_yesom\n✉︎Business contac...,,2021-05-15,None,"[파츠, 소분, 하, 고, 비즈, 키트, 제작, 하, 는, 쇼핑몰, 브이, 로, 그...",1


In [105]:
data[['video_id','title','predict']].to_csv('FastTEXT_result2.csv',encoding='utf-8-sig',index=False)

In [49]:
df_topic = data[['video_id','channel_id','predict']]

In [46]:
data['predict'].value_counts()

8     277641
15    157017
11    126138
1      91728
2      88277
7      87486
9      85621
5      75179
14     73270
0      50207
16     49015
13     44661
4      42562
10     34311
6      31330
12     31039
3      20435
17      1317
Name: predict, dtype: int64

In [47]:
y

array([[1.0701390e-02, 2.1562683e-03, 4.0774518e-03, ..., 2.0338224e-04,
        1.8592521e-04, 3.5809033e-04],
       [9.5250661e-04, 1.4375028e-01, 8.5118851e-03, ..., 1.3605281e-03,
        7.5070030e-04, 1.7818686e-03],
       [3.1823695e-03, 4.2525183e-02, 9.2313811e-02, ..., 4.3316479e-03,
        5.5069998e-02, 1.3619551e-02],
       ...,
       [8.5484089e-06, 2.2795368e-05, 2.1242786e-06, ..., 6.8294110e-05,
        1.7179961e-08, 1.8119597e-06],
       [1.7179900e-03, 4.3278746e-02, 6.6912514e-01, ..., 3.1289936e-03,
        1.9907481e-03, 1.1907844e-02],
       [8.1063365e-04, 1.9788144e-03, 3.5791658e-04, ..., 2.8964174e-03,
        2.9501042e-03, 3.8595318e-03]], dtype=float32)

In [107]:
cate_columns = ['가전제품', '자동차와 오토바이', '여행', '동물', '패션뷰티', '게임',
       '생활리빙', '음악', '운동과 스포츠', '예술과 인문학', '미디어 콘텐츠', '언론과 사회', '과학과 기술',
       '건강과 의료', '돈과 성공', '음식', '종교', '기타']

In [108]:
df_yt_movie_result = pd.concat([df_topic,pd.DataFrame(y,columns=cate_columns)],1)

<ipython-input-108-212eefbc2c73>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_yt_movie_result = pd.concat([df_topic,pd.DataFrame(y,columns=cate_columns)],1)


In [109]:
df_yt_movie_result

,video_id,channel_id,predict,가전제품,자동차와 오토바이,여행,동물,패션뷰티,게임,생활리빙,...,운동과 스포츠,예술과 인문학,미디어 콘텐츠,언론과 사회,과학과 기술,건강과 의료,돈과 성공,음식,종교,기타
0,___CMA1pEXA,UC12cVwwjEC1CG4Y8abhr9Xg,7,0.000032,0.000235,0.000064,0.000112,0.000878,0.000057,0.000024,...,0.003583,0.000539,0.006831,0.001562,0.001170,0.002837,0.000063,0.000169,0.000376,0.000010
1,___m5buSuTk,UCcRy0bDlIh0sKWiOv7XWQmA,13,0.000170,0.000578,0.000246,0.000491,0.002441,0.000124,0.000519,...,0.003586,0.000378,0.000452,0.010320,0.058172,0.917112,0.000526,0.000607,0.002690,0.000063
2,___M9uxeTPo,UCksyDD1Gv6wmpyXJezZkVXw,11,0.000325,0.003323,0.000216,0.000182,0.000066,0.000139,0.000534,...,0.001539,0.001875,0.000725,0.974166,0.004325,0.002135,0.000969,0.000558,0.007415,0.001355
3,___pCwlibvw,UCSuuVcF9lix5dwFmAPkHE-g,9,0.000448,0.015403,0.001741,0.002543,0.002191,0.005004,0.006261,...,0.017583,0.500588,0.126662,0.114867,0.161906,0.007826,0.002593,0.016086,0.010493,0.004267
4,__-5OLdwYaw,UCeXeCiZZ5OtiBTTn7UNN-cQ,7,0.000081,0.000436,0.000342,0.000038,0.001378,0.000712,0.000087,...,0.006222,0.000815,0.000474,0.002693,0.002035,0.000952,0.000283,0.000131,0.001047,0.000028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367229,ZzZyzYp8m9w,UCAnRSFRuxYU087HNOGta3BA,2,0.000138,0.003708,0.984681,0.000102,0.000118,0.000021,0.000174,...,0.000360,0.000181,0.002066,0.006482,0.000016,0.001173,0.000122,0.000211,0.000141,0.000041
1367230,ZzZZEbhJOi8,UCe8v7Loack1KSmvCpDhVv1w,8,0.000463,0.007013,0.000250,0.000036,0.003145,0.003621,0.000086,...,0.660289,0.291225,0.014888,0.006078,0.000906,0.000111,0.001178,0.000699,0.003380,0.000188
1367231,zzzzjjJ69Ew,UCRwrZJ9OCEIvdXFz0AZ5KNw,8,0.000041,0.000143,0.000018,0.000007,0.000063,0.000008,0.000020,...,0.998875,0.000060,0.000062,0.000228,0.000018,0.000014,0.000173,0.000218,0.000022,0.000015
1367232,zZzZoEbxZK0,UCmxawXaLxT08isee1XTPXUg,2,0.000467,0.504201,0.072947,0.001428,0.023675,0.009304,0.110157,...,0.038774,0.003933,0.053202,0.107507,0.012378,0.000855,0.000977,0.014677,0.003560,0.041788


In [110]:
channel_vector = df_yt_movie_result.groupby('channel_id').sum()

In [111]:
channel_vector = channel_vector.drop(['predict'],1)
channel_vector

<ipython-input-111-c3e81bdd0c60>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  channel_vector = channel_vector.drop(['predict'],1)


,가전제품,자동차와 오토바이,여행,동물,패션뷰티,게임,생활리빙,음악,운동과 스포츠,예술과 인문학,미디어 콘텐츠,언론과 사회,과학과 기술,건강과 의료,돈과 성공,음식,종교,기타
channel_id,,,,,,,,,,,,,,,,,,
UC--8ua5dEkuY26fgWkpkz3Q,0.045507,2.801092,5.100846,0.201750,0.791068,0.493956,0.105033,2.004661,1.708216,0.218930,3.233646,5.928011,0.629334,2.763227,0.190749,2.770147,0.605392,0.408434
UC--GbJkv6N0LLuUWeZDaXvw,0.372161,1.153450,0.747961,0.189959,0.255679,0.982103,0.174482,4.618145,4.387710,2.180858,1.770660,1.563973,4.273566,1.533992,0.939534,2.705045,2.000024,0.150697
UC--L--_f3e5In69IyzMPLRg,0.007348,0.289573,0.020494,0.006789,0.005501,0.037994,0.020195,0.004438,0.209254,0.683859,0.097807,0.519008,2.286097,0.079592,0.127987,0.113576,25.464840,0.025648
UC--LgKcZVgffjsxudoXg5pQ,0.778847,0.378478,0.070499,0.033318,0.042845,0.248498,0.900140,0.273214,9.554596,1.228400,0.289325,2.157008,10.565044,0.224638,1.692865,0.710191,0.761633,0.090460
UC--McyqJNLmOx2qpdT5SkvQ,0.016105,0.019699,0.000632,0.000484,0.000690,28.581451,0.001802,0.005556,0.020442,0.620117,0.015424,0.082711,0.446423,0.001065,0.004134,0.106998,0.048806,0.027461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UCzzsPLVj67qyXrVcGmewSmg,0.097596,0.186419,0.037054,0.030376,0.048763,0.093968,0.094638,0.031768,16.287380,0.509826,0.068222,1.339800,8.712616,0.395587,0.143825,1.161147,0.696306,0.064708
UCzzt6bFEBcy-CoZeFvTSq0A,1.012780,2.804152,1.775112,0.306493,4.027022,0.214814,0.497362,0.110746,1.994403,0.138428,1.693549,2.626848,2.993192,3.020907,0.093806,4.975983,1.330644,0.383760
UCzzt9YrAmPsya3zBMmS9Nmw,1.008323,0.669825,0.081638,0.008967,0.105560,8.816328,0.014600,0.021062,11.083238,0.015948,0.032223,1.361844,5.300287,0.196258,0.073783,0.444987,0.437175,0.327954


In [112]:
channel_vector_prop = channel_vector.div(channel_vector.sum(axis=1), axis=0)
channel_vector_prop

,가전제품,자동차와 오토바이,여행,동물,패션뷰티,게임,생활리빙,음악,운동과 스포츠,예술과 인문학,미디어 콘텐츠,언론과 사회,과학과 기술,건강과 의료,돈과 성공,음식,종교,기타
channel_id,,,,,,,,,,,,,,,,,,
UC--8ua5dEkuY26fgWkpkz3Q,0.001517,0.093370,0.170028,0.006725,0.026369,0.016465,0.003501,0.066822,0.056941,0.007298,0.107788,0.197600,0.020978,0.092108,0.006358,0.092338,0.020180,0.013614
UC--GbJkv6N0LLuUWeZDaXvw,0.012405,0.038448,0.024932,0.006332,0.008523,0.032737,0.005816,0.153938,0.146257,0.072695,0.059022,0.052132,0.142452,0.051133,0.031318,0.090168,0.066667,0.005023
UC--L--_f3e5In69IyzMPLRg,0.000245,0.009652,0.000683,0.000226,0.000183,0.001266,0.000673,0.000148,0.006975,0.022795,0.003260,0.017300,0.076203,0.002653,0.004266,0.003786,0.848828,0.000855
UC--LgKcZVgffjsxudoXg5pQ,0.025962,0.012616,0.002350,0.001111,0.001428,0.008283,0.030005,0.009107,0.318487,0.040947,0.009644,0.071900,0.352168,0.007488,0.056429,0.023673,0.025388,0.003015
UC--McyqJNLmOx2qpdT5SkvQ,0.000537,0.000657,0.000021,0.000016,0.000023,0.952715,0.000060,0.000185,0.000681,0.020671,0.000514,0.002757,0.014881,0.000035,0.000138,0.003567,0.001627,0.000915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UCzzsPLVj67qyXrVcGmewSmg,0.003253,0.006214,0.001235,0.001013,0.001625,0.003132,0.003155,0.001059,0.542913,0.016994,0.002274,0.044660,0.290421,0.013186,0.004794,0.038705,0.023210,0.002157
UCzzt6bFEBcy-CoZeFvTSq0A,0.033759,0.093472,0.059170,0.010216,0.134234,0.007160,0.016579,0.003692,0.066480,0.004614,0.056452,0.087562,0.099773,0.100697,0.003127,0.165866,0.044355,0.012792
UCzzt9YrAmPsya3zBMmS9Nmw,0.033611,0.022328,0.002721,0.000299,0.003519,0.293878,0.000487,0.000702,0.369441,0.000532,0.001074,0.045395,0.176676,0.006542,0.002459,0.014833,0.014573,0.010932


In [113]:
channel_vector_prop['channel_id'] = channel_vector_prop.index
channel_vector_prop.reset_index(drop=True,inplace=True)

In [114]:
third_highest_value_index = channel_vector_prop[cate_columns].apply(lambda s: s.nlargest(3).index.tolist(), axis=1)

In [115]:
df_rank = pd.DataFrame(third_highest_value_index.tolist(), columns=['first_cate','second_cate','third_cate'])

In [116]:
df_channel_insert = pd.concat([channel_vector_prop,df_rank],1)
df_channel_insert

<ipython-input-116-67c87596cb87>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_channel_insert = pd.concat([channel_vector_prop,df_rank],1)


,가전제품,자동차와 오토바이,여행,동물,패션뷰티,게임,생활리빙,음악,운동과 스포츠,예술과 인문학,...,과학과 기술,건강과 의료,돈과 성공,음식,종교,기타,channel_id,first_cate,second_cate,third_cate
0,0.001517,0.093370,0.170028,0.006725,0.026369,0.016465,0.003501,0.066822,0.056941,0.007298,...,0.020978,0.092108,0.006358,0.092338,0.020180,0.013614,UC--8ua5dEkuY26fgWkpkz3Q,언론과 사회,여행,미디어 콘텐츠
1,0.012405,0.038448,0.024932,0.006332,0.008523,0.032737,0.005816,0.153938,0.146257,0.072695,...,0.142452,0.051133,0.031318,0.090168,0.066667,0.005023,UC--GbJkv6N0LLuUWeZDaXvw,음악,운동과 스포츠,과학과 기술
2,0.000245,0.009652,0.000683,0.000226,0.000183,0.001266,0.000673,0.000148,0.006975,0.022795,...,0.076203,0.002653,0.004266,0.003786,0.848828,0.000855,UC--L--_f3e5In69IyzMPLRg,종교,과학과 기술,예술과 인문학
3,0.025962,0.012616,0.002350,0.001111,0.001428,0.008283,0.030005,0.009107,0.318487,0.040947,...,0.352168,0.007488,0.056429,0.023673,0.025388,0.003015,UC--LgKcZVgffjsxudoXg5pQ,과학과 기술,운동과 스포츠,언론과 사회
4,0.000537,0.000657,0.000021,0.000016,0.000023,0.952715,0.000060,0.000185,0.000681,0.020671,...,0.014881,0.000035,0.000138,0.003567,0.001627,0.000915,UC--McyqJNLmOx2qpdT5SkvQ,게임,예술과 인문학,과학과 기술
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48321,0.003253,0.006214,0.001235,0.001013,0.001625,0.003132,0.003155,0.001059,0.542913,0.016994,...,0.290421,0.013186,0.004794,0.038705,0.023210,0.002157,UCzzsPLVj67qyXrVcGmewSmg,운동과 스포츠,과학과 기술,언론과 사회
48322,0.033759,0.093472,0.059170,0.010216,0.134234,0.007160,0.016579,0.003692,0.066480,0.004614,...,0.099773,0.100697,0.003127,0.165866,0.044355,0.012792,UCzzt6bFEBcy-CoZeFvTSq0A,음식,패션뷰티,건강과 의료
48323,0.033611,0.022328,0.002721,0.000299,0.003519,0.293878,0.000487,0.000702,0.369441,0.000532,...,0.176676,0.006542,0.002459,0.014833,0.014573,0.010932,UCzzt9YrAmPsya3zBMmS9Nmw,운동과 스포츠,게임,과학과 기술
48324,0.011244,0.078432,0.005434,0.004264,0.000609,0.000303,0.137212,0.000649,0.012785,0.016606,...,0.022049,0.135193,0.253863,0.014974,0.012727,0.001983,UCzztPvTc6mfxKji1AkyROgw,언론과 사회,돈과 성공,생활리빙


In [129]:
df_channel_insert[120:140]

,channel_id,electronic,car,trip,animal,beauty,game,living,music,sports,...,society,science,health,money,food,religion,etc,main_cate,second_cate,third_cate
120,UC-9YizJjYnGONHh5jRSXE6Q,1.615574e-08,0.051079,0.000005,3.795294e-07,1.785327e-08,1.396057e-07,0.000002,0.000014,0.000016,...,0.000048,1.068614e-07,0.000038,0.000004,0.000001,0.948777,0.000008,종교,자동차와 오토바이,언론과 사회
121,UC-9icJsID5WLwHEToM0QYGA,7.423208e-03,0.027195,0.004008,1.676410e-03,1.016330e-03,9.333169e-04,0.022701,0.001658,0.025398,...,0.085913,1.028266e-01,0.017520,0.639104,0.024773,0.025471,0.002661,돈과 성공,과학과 기술,언론과 사회
122,UC-9kfEMkO7Zhbre7DJtaP3w,9.608989e-03,0.020910,0.086233,4.229661e-02,1.978691e-01,5.120597e-02,0.097450,0.082670,0.034928,...,0.022922,8.371742e-02,0.004213,0.001972,0.062795,0.004637,0.006066,패션뷰티,,
123,UC-9n4pEUltxpG5lOyZUOn6Q,5.464737e-03,0.038582,0.005852,3.555680e-03,2.250522e-02,5.475569e-03,0.005242,0.288990,0.161994,...,0.151309,8.768523e-03,0.008151,0.020957,0.115920,0.021052,0.003992,음악,운동과 스포츠,
124,UC-9nTjw_qHCA693UA3jyDWw,1.399793e-01,0.029819,0.003490,7.894104e-03,2.127796e-02,1.203483e-02,0.020207,0.228640,0.134714,...,0.040506,3.080822e-02,0.019155,0.123101,0.132860,0.011040,0.005692,음악,가전제품,
125,UC-9oz6OofrZnaOUQ48-LavA,2.445248e-04,0.002442,0.000951,3.269763e-04,1.930484e-03,5.881999e-04,0.002216,0.000103,0.022987,...,0.002704,6.322040e-04,0.012669,0.000686,0.949777,0.000529,0.000579,음식,운동과 스포츠,건강과 의료
126,UC-A8G5BC2gt2HAAsJpSVwhw,2.057260e-02,0.043450,0.139786,8.153958e-03,2.295789e-03,1.252798e-02,0.008147,0.002084,0.159355,...,0.083544,1.617560e-01,0.014063,0.006206,0.096629,0.097329,0.008716,과학과 기술,,
127,UC-ALJHclOi2SioUH2aVlvAQ,1.454824e-02,0.039732,0.017211,2.635755e-03,2.414313e-02,2.976739e-03,0.013197,0.001971,0.030708,...,0.121872,3.266100e-01,0.018879,0.236514,0.015452,0.093914,0.005675,과학과 기술,돈과 성공,언론과 사회
128,UC-AMEZEYu-k6pRwWfDS7N3w,1.211932e-02,0.029513,0.011185,1.170169e-02,1.884334e-02,7.980716e-03,0.155735,0.007182,0.109387,...,0.037598,2.933997e-01,0.010441,0.024905,0.026143,0.168285,0.004378,과학과 기술,종교,
129,UC-AO0cgzfFicEep6MtXDQGQ,5.502949e-03,0.007957,0.002383,6.282808e-03,2.497660e-03,3.383537e-03,0.005604,0.003535,0.014184,...,0.045594,1.418004e-01,0.015552,0.010035,0.098032,0.617767,0.007498,종교,과학과 기술,음식


In [60]:
df_channel_insert['first_cate'].value_counts()

운동과 스포츠      12469
음식            5752
언론과 사회        4139
음악            3686
게임            2979
여행            2862
돈과 성공         2777
예술과 인문학       2625
자동차와 오토바이     2124
패션뷰티          1610
종교            1439
가전제품          1252
건강과 의료        1222
미디어 콘텐츠       1137
생활리빙           954
동물             767
과학과 기술         527
기타               5
Name: first_cate, dtype: int64

In [117]:
df_channel_insert['first_cate'].value_counts()

음악           6667
운동과 스포츠      6159
과학과 기술       6080
언론과 사회       5514
음식           3302
게임           2797
돈과 성공        2415
패션뷰티         2395
미디어 콘텐츠      2162
종교           1888
자동차와 오토바이    1828
예술과 인문학      1317
생활리빙         1302
건강과 의료       1291
여행           1188
가전제품         1034
동물            987
Name: first_cate, dtype: int64

In [61]:
cursor.execute("Select * FROM YT_channel_cate") #From 367
channel_columns = [item[0] for item in cursor.description]

In [118]:
channel_columns

['cc_no',
 'channel_id',
 'electronic',
 'car',
 'trip',
 'animal',
 'beauty',
 'game',
 'living',
 'music',
 'sports',
 'art',
 'media',
 'society',
 'science',
 'health',
 'money',
 'food',
 'religion',
 'etc',
 'main_cate',
 'second_cate',
 'third_cate']

In [119]:
df_channel_insert.columns = ['electronic', 'car', 'trip', 'animal', 'beauty', 'game', 'living',
 'music', 'sports', 'art', 'media', 'society', 'science', 'health', 'money', 'food', 'religion',
 'etc','channel_id','main_cate', 'second_cate', 'third_cate']

In [120]:
df_channel_insert

,electronic,car,trip,animal,beauty,game,living,music,sports,art,...,science,health,money,food,religion,etc,channel_id,main_cate,second_cate,third_cate
0,0.001517,0.093370,0.170028,0.006725,0.026369,0.016465,0.003501,0.066822,0.056941,0.007298,...,0.020978,0.092108,0.006358,0.092338,0.020180,0.013614,UC--8ua5dEkuY26fgWkpkz3Q,언론과 사회,여행,미디어 콘텐츠
1,0.012405,0.038448,0.024932,0.006332,0.008523,0.032737,0.005816,0.153938,0.146257,0.072695,...,0.142452,0.051133,0.031318,0.090168,0.066667,0.005023,UC--GbJkv6N0LLuUWeZDaXvw,음악,운동과 스포츠,과학과 기술
2,0.000245,0.009652,0.000683,0.000226,0.000183,0.001266,0.000673,0.000148,0.006975,0.022795,...,0.076203,0.002653,0.004266,0.003786,0.848828,0.000855,UC--L--_f3e5In69IyzMPLRg,종교,과학과 기술,예술과 인문학
3,0.025962,0.012616,0.002350,0.001111,0.001428,0.008283,0.030005,0.009107,0.318487,0.040947,...,0.352168,0.007488,0.056429,0.023673,0.025388,0.003015,UC--LgKcZVgffjsxudoXg5pQ,과학과 기술,운동과 스포츠,언론과 사회
4,0.000537,0.000657,0.000021,0.000016,0.000023,0.952715,0.000060,0.000185,0.000681,0.020671,...,0.014881,0.000035,0.000138,0.003567,0.001627,0.000915,UC--McyqJNLmOx2qpdT5SkvQ,게임,예술과 인문학,과학과 기술
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48321,0.003253,0.006214,0.001235,0.001013,0.001625,0.003132,0.003155,0.001059,0.542913,0.016994,...,0.290421,0.013186,0.004794,0.038705,0.023210,0.002157,UCzzsPLVj67qyXrVcGmewSmg,운동과 스포츠,과학과 기술,언론과 사회
48322,0.033759,0.093472,0.059170,0.010216,0.134234,0.007160,0.016579,0.003692,0.066480,0.004614,...,0.099773,0.100697,0.003127,0.165866,0.044355,0.012792,UCzzt6bFEBcy-CoZeFvTSq0A,음식,패션뷰티,건강과 의료
48323,0.033611,0.022328,0.002721,0.000299,0.003519,0.293878,0.000487,0.000702,0.369441,0.000532,...,0.176676,0.006542,0.002459,0.014833,0.014573,0.010932,UCzzt9YrAmPsya3zBMmS9Nmw,운동과 스포츠,게임,과학과 기술
48324,0.011244,0.078432,0.005434,0.004264,0.000609,0.000303,0.137212,0.000649,0.012785,0.016606,...,0.022049,0.135193,0.253863,0.014974,0.012727,0.001983,UCzztPvTc6mfxKji1AkyROgw,언론과 사회,돈과 성공,생활리빙


In [121]:
df_channel_insert = df_channel_insert[['channel_id','electronic',
 'car',
 'trip',
 'animal',
 'beauty',
 'game',
 'living',
 'music',
 'sports',
 'art',
 'media',
 'society',
 'science',
 'health',
 'money',
 'food',
 'religion',
 'etc','main_cate',
 'second_cate',
 'third_cate']]

In [122]:
df_channel_insert

,channel_id,electronic,car,trip,animal,beauty,game,living,music,sports,...,society,science,health,money,food,religion,etc,main_cate,second_cate,third_cate
0,UC--8ua5dEkuY26fgWkpkz3Q,0.001517,0.093370,0.170028,0.006725,0.026369,0.016465,0.003501,0.066822,0.056941,...,0.197600,0.020978,0.092108,0.006358,0.092338,0.020180,0.013614,언론과 사회,여행,미디어 콘텐츠
1,UC--GbJkv6N0LLuUWeZDaXvw,0.012405,0.038448,0.024932,0.006332,0.008523,0.032737,0.005816,0.153938,0.146257,...,0.052132,0.142452,0.051133,0.031318,0.090168,0.066667,0.005023,음악,운동과 스포츠,과학과 기술
2,UC--L--_f3e5In69IyzMPLRg,0.000245,0.009652,0.000683,0.000226,0.000183,0.001266,0.000673,0.000148,0.006975,...,0.017300,0.076203,0.002653,0.004266,0.003786,0.848828,0.000855,종교,과학과 기술,예술과 인문학
3,UC--LgKcZVgffjsxudoXg5pQ,0.025962,0.012616,0.002350,0.001111,0.001428,0.008283,0.030005,0.009107,0.318487,...,0.071900,0.352168,0.007488,0.056429,0.023673,0.025388,0.003015,과학과 기술,운동과 스포츠,언론과 사회
4,UC--McyqJNLmOx2qpdT5SkvQ,0.000537,0.000657,0.000021,0.000016,0.000023,0.952715,0.000060,0.000185,0.000681,...,0.002757,0.014881,0.000035,0.000138,0.003567,0.001627,0.000915,게임,예술과 인문학,과학과 기술
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48321,UCzzsPLVj67qyXrVcGmewSmg,0.003253,0.006214,0.001235,0.001013,0.001625,0.003132,0.003155,0.001059,0.542913,...,0.044660,0.290421,0.013186,0.004794,0.038705,0.023210,0.002157,운동과 스포츠,과학과 기술,언론과 사회
48322,UCzzt6bFEBcy-CoZeFvTSq0A,0.033759,0.093472,0.059170,0.010216,0.134234,0.007160,0.016579,0.003692,0.066480,...,0.087562,0.099773,0.100697,0.003127,0.165866,0.044355,0.012792,음식,패션뷰티,건강과 의료
48323,UCzzt9YrAmPsya3zBMmS9Nmw,0.033611,0.022328,0.002721,0.000299,0.003519,0.293878,0.000487,0.000702,0.369441,...,0.045395,0.176676,0.006542,0.002459,0.014833,0.014573,0.010932,운동과 스포츠,게임,과학과 기술
48324,UCzztPvTc6mfxKji1AkyROgw,0.011244,0.078432,0.005434,0.004264,0.000609,0.000303,0.137212,0.000649,0.012785,...,0.290908,0.022049,0.135193,0.253863,0.014974,0.012727,0.001983,언론과 사회,돈과 성공,생활리빙


In [75]:
eng_num_col = ['electronic',
 'car',
 'trip',
 'animal',
 'beauty',
 'game',
 'living',
 'music',
 'sports',
 'art',
 'media',
 'society',
 'science',
 'health',
 'money',
 'food',
 'religion',
 'etc']

In [123]:
df_channel_insert.loc[df_channel_insert[df_channel_insert[eng_num_col].max(axis="columns")<0.3].index,'third_cate'] = ''
df_channel_insert.loc[df_channel_insert[df_channel_insert[eng_num_col].max(axis="columns")<0.2].index,'second_cate'] = ''
df_channel_insert.loc[df_channel_insert[df_channel_insert[eng_num_col].max(axis="columns")<0.1].index,'main_cate'] = '기타'

In [124]:
df_channel_insert

,channel_id,electronic,car,trip,animal,beauty,game,living,music,sports,...,society,science,health,money,food,religion,etc,main_cate,second_cate,third_cate
0,UC--8ua5dEkuY26fgWkpkz3Q,0.001517,0.093370,0.170028,0.006725,0.026369,0.016465,0.003501,0.066822,0.056941,...,0.197600,0.020978,0.092108,0.006358,0.092338,0.020180,0.013614,언론과 사회,,
1,UC--GbJkv6N0LLuUWeZDaXvw,0.012405,0.038448,0.024932,0.006332,0.008523,0.032737,0.005816,0.153938,0.146257,...,0.052132,0.142452,0.051133,0.031318,0.090168,0.066667,0.005023,음악,,
2,UC--L--_f3e5In69IyzMPLRg,0.000245,0.009652,0.000683,0.000226,0.000183,0.001266,0.000673,0.000148,0.006975,...,0.017300,0.076203,0.002653,0.004266,0.003786,0.848828,0.000855,종교,과학과 기술,예술과 인문학
3,UC--LgKcZVgffjsxudoXg5pQ,0.025962,0.012616,0.002350,0.001111,0.001428,0.008283,0.030005,0.009107,0.318487,...,0.071900,0.352168,0.007488,0.056429,0.023673,0.025388,0.003015,과학과 기술,운동과 스포츠,언론과 사회
4,UC--McyqJNLmOx2qpdT5SkvQ,0.000537,0.000657,0.000021,0.000016,0.000023,0.952715,0.000060,0.000185,0.000681,...,0.002757,0.014881,0.000035,0.000138,0.003567,0.001627,0.000915,게임,예술과 인문학,과학과 기술
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48321,UCzzsPLVj67qyXrVcGmewSmg,0.003253,0.006214,0.001235,0.001013,0.001625,0.003132,0.003155,0.001059,0.542913,...,0.044660,0.290421,0.013186,0.004794,0.038705,0.023210,0.002157,운동과 스포츠,과학과 기술,언론과 사회
48322,UCzzt6bFEBcy-CoZeFvTSq0A,0.033759,0.093472,0.059170,0.010216,0.134234,0.007160,0.016579,0.003692,0.066480,...,0.087562,0.099773,0.100697,0.003127,0.165866,0.044355,0.012792,음식,,
48323,UCzzt9YrAmPsya3zBMmS9Nmw,0.033611,0.022328,0.002721,0.000299,0.003519,0.293878,0.000487,0.000702,0.369441,...,0.045395,0.176676,0.006542,0.002459,0.014833,0.014573,0.010932,운동과 스포츠,게임,과학과 기술
48324,UCzztPvTc6mfxKji1AkyROgw,0.011244,0.078432,0.005434,0.004264,0.000609,0.000303,0.137212,0.000649,0.012785,...,0.290908,0.022049,0.135193,0.253863,0.014974,0.012727,0.001983,언론과 사회,돈과 성공,


In [88]:
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import MySQLdb


In [89]:
engine = create_engine("mysql+mysqldb://"+user+":"+str(passwd)+"@"+host+":"+str(port)+"/"+db, encoding='utf-8')


In [90]:
df_channel_insert.to_sql(name='YT_channel_cate', con=engine, if_exists='append', index=False)